In [ ]:
versioninfo()

# New version of main

- [X] refactor the code of main1
- [X] use seeding
- [X] pre-gen graphs and splitting
- [X] use cloud GPU for training
- [X] I don't need correlation experiments anymore

In [ ]:
include("main.jl")

In [ ]:
main()

In [ ]:
main_ensemble()

# Sachs 2005 experiment

## Load and explore the data

In [ ]:
import CSV

In [ ]:
df = CSV.read("Sachs/1.cd3cd28.csv")

In [ ]:
SachsX = convert(Matrix, df)

In [ ]:
include("data_graph.jl")

In [ ]:
SachsG = Sachs_ground_truth()

In [ ]:
myplot(SachsG)

In [ ]:
medcovX = cov(SachsX) ./ median(var(SachsX, dims=1))

In [ ]:
maxcovX = cov(SachsX) ./ maximum(var(SachsX, dims=1))

In [ ]:
corX = cor(SachsX)

In [ ]:
include("exp.jl")

## Load a model

In [ ]:
# load the trained model
@load "saved_models/EQ-d=20_k=1_gtype=SF_noise=Gaussian_mat=medCOV_mec=Linear/step-15000.bson" model

In [ ]:
@load "saved_models/EQ-d=10_k=1_gtype=ER_noise=Gaussian_mat=medCOV_mec=Linear/step-15000.bson" model

In [ ]:
@load "back/back-0907/CORCOV/EQ-d=10_k=1_gtype=SF_noise=Gaussian_mat=COR_mec=Linear/step-15000.bson" model

In [ ]:
@load "saved_models/ensK-2020-09-08T10:58:41.247-ensemble/step-10000.bson" model

## Evaluate the model

In [ ]:
out = inf_one(model, medcovX)

In [ ]:
out = inf_one(model, corX)

In [ ]:
out = inf_one(model, maxcovX)

In [ ]:
Wout = threshold(σ.(out), 0.5, true)

In [ ]:
myplot(DiGraph(Wout), names(df))

In [ ]:
myplot(SachsG, names(df))

In [ ]:
# predicted edge, true edge, SHD
predicted_edge = ne(DiGraph(Wout))
@show predicted_edge
correct_edge = sum(Wout[Wout .== 1] .== adjacency_matrix(SachsG)[Wout .== 1])
@show correct_edge

# metrics
ytrue = Matrix(gen_weights(SachsG))
sup_graph_metrics(Wout, ytrue)